### Multinomial classification

transform scores(logit) into probabilities by softmax
- sum of probability = 1

In [1]:
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Data
x_data = [[1,2,1,1],[2,1,3,2],[3,1,3,4],[4,1,5,5],[1,7,5,5],[1,2,5,6],[1,6,6,6],[1,7,7,7]]
y_data = [[0,0,1],[0,0,1],[0,0,1],[0,1,0],[0,1,0],[0,1,0],[1,0,0],[1,0,0]]

x_dim = len(x_data[0])
y_dim = len(y_data[0])

In [3]:
x_dim, y_dim

(4, 3)

In [4]:
X = tf.placeholder('float',[None, x_dim])
Y = tf.placeholder('float',[None, y_dim])
nb_classes =3

In [5]:
W = tf.Variable(tf.random_normal([x_dim, nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')

In [6]:
#tf.nn.softmax computes softmax function
#softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X,W) + b)

In [7]:
#Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis = 1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

In [8]:
#Launch graph

config = tf.ConfigProto(device_count={'GPU': 0}) #gpu error 발생
with tf.Session(config = config) as sess:
    sess.run(tf.global_variables_initializer())
    feed = {X:x_data, Y:y_data}
    
    for step in range(2001):
        sess.run(optimizer, feed_dict =feed)
        
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict = feed))
            
    #Test & one-hot encoding
    a = sess.run(hypothesis, feed_dict =feed)
    print('\n',a, sess.run(tf.argmax(a,1))) #argmax: a의 몇 번째 값이 제일 큰 지. axis = 1

0 5.350521
200 0.57921684
400 0.4683261
600 0.37240213
800 0.28091782
1000 0.23318046
1200 0.21140188
1400 0.1932642
1600 0.1779136
1800 0.16475494
2000 0.15335369

 [[2.5528173e-06 1.1735437e-03 9.9882394e-01]
 [2.0594189e-03 8.0468521e-02 9.1747212e-01]
 [1.5066603e-07 1.5990530e-01 8.4009457e-01]
 [6.2314680e-06 8.5350978e-01 1.4648405e-01]
 [2.5081733e-01 7.3727888e-01 1.1903838e-02]
 [1.3172181e-01 8.6824757e-01 3.0515812e-05]
 [7.5479293e-01 2.4517184e-01 3.5154324e-05]
 [9.2357683e-01 7.6422684e-02 5.7871125e-07]] [2 2 2 1 1 1 0 0]


## Fancy Softmax Classifier
- corss_entropy, one_hot, reshape function

In [9]:
import numpy as np

xy = np.loadtxt('data-04-zoo.csv', delimiter = ',', dtype = np.float32)
x_data = xy[:,:-1]
y_data = xy[:,[-1]]

In [10]:
x_dim = len(x_data[0])
nb_classes = 7  # 0~6

In [11]:
X = tf.placeholder(tf.float32, [None, x_dim])
Y = tf.placeholder(tf.int32, [None, 1])
Y_one_hot = tf.one_hot(Y, nb_classes)  #one hot shape = (?, 1, 7)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) #shape = (?, 7)

In [12]:
W = tf.Variable(tf.random_normal([x_dim, nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')

#### 첫 번째 one_hot으로 돌리면 에러 발생!
If the input indices is rank N, the output will have rank N+1. 

EX) [[0],[3]]  => [[[1,0,0,0,0,0],[[0,0,0,1,0,0]]

In [13]:
#softmax_cross_entropy_with_logits

#before: cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis = 1))
#NEW!!!
logits = tf.matmul(X,W) + b
hypothesis = tf.nn.softmax(logits)

cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y_one_hot)
cost = tf.reduce_mean(cost_i)

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [14]:
#우리의 모델이 맞았는지 틀렸는지 확인하기 위한 것들
prediction = tf.argmax(hypothesis, 1) #0~6의 값을 출력(가장 값이 큰 class)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1)) #예측이 맞으면 T
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #T인 값들을 1로!

In [18]:
config = tf.ConfigProto(device_count={'GPU': 0}) #gpu error 발생

with tf.Session(config = config) as sess:
    sess.run(tf.global_variables_initializer())
    feed = {X:x_data, Y:y_data}
    
    for step in range(2000):
        sess.run(optimizer, feed_dict = feed)
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict = feed)
            print('Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}'.format(step, loss, acc))
            
    pred = sess.run(prediction, feed_dict = feed)
    #y_data: (N,1)  == flatten ==> (N, ) matches pred.shpe
    for p, y in zip(pred, y_data.flatten()):
        print('[{}] Prediction: {} True Y: {}'.format(p == int(y), p, int(y)))

Step:     0	Loss: 3.373	Acc: 27.72%
Step:   100	Loss: 0.727	Acc: 71.29%
Step:   200	Loss: 0.417	Acc: 85.15%
Step:   300	Loss: 0.284	Acc: 95.05%
Step:   400	Loss: 0.215	Acc: 97.03%
Step:   500	Loss: 0.173	Acc: 99.01%
Step:   600	Loss: 0.145	Acc: 100.00%
Step:   700	Loss: 0.125	Acc: 100.00%
Step:   800	Loss: 0.110	Acc: 100.00%
Step:   900	Loss: 0.098	Acc: 100.00%
Step:  1000	Loss: 0.089	Acc: 100.00%
Step:  1100	Loss: 0.081	Acc: 100.00%
Step:  1200	Loss: 0.075	Acc: 100.00%
Step:  1300	Loss: 0.070	Acc: 100.00%
Step:  1400	Loss: 0.065	Acc: 100.00%
Step:  1500	Loss: 0.061	Acc: 100.00%
Step:  1600	Loss: 0.058	Acc: 100.00%
Step:  1700	Loss: 0.054	Acc: 100.00%
Step:  1800	Loss: 0.052	Acc: 100.00%
Step:  1900	Loss: 0.049	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: